In [8]:
import spacy
import pandas as pd
from rake_nltk import Rake
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\BlackLine\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
# Load your job descriptions
data = [
    "We are looking for a Data Scientist with expertise in Python, machine learning, and SQL.",
    "Frontend developer required with knowledge of React, JavaScript, HTML, and CSS.",
    "Experience with AWS cloud technologies, Docker, and Kubernetes is a must."
]

df = pd.DataFrame({'job_description': data})

In [10]:
def preprocess(text):
    # Remove special characters and lowercase
    text = re.sub(r'[^\w\s]', '', text.lower())
    return text

df['cleaned'] = df['job_description'].apply(preprocess)


In [18]:
nltk.download('punkt')
nltk.download('punkt_tab')

rake = Rake()

def extract_keywords_rake(text):
    rake.extract_keywords_from_text(text)
    return rake.get_ranked_phrases()

df['rake_keywords'] = df['cleaned'].apply(extract_keywords_rake)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\BlackLine\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\BlackLine\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


In [19]:
vectorizer = TfidfVectorizer(max_features=10)
tfidf_matrix = vectorizer.fit_transform(df['cleaned'])
tfidf_keywords = vectorizer.get_feature_names_out()

df['tfidf_keywords'] = [tfidf_keywords for _ in range(len(df))]


In [25]:
nlp = spacy.load("en_core_web_sm")

def extract_spacy_entities(text):
    doc = nlp(text)
    return [(ent.text, ent.label_) for ent in doc.ents]

df['spacy_entities'] = df['job_description'].apply(extract_spacy_entities)


In [39]:
# Install PyTorch
%pip install torch
%pip install transformers

import torch
from transformers import pipeline

# Load a pre-trained pipeline for token classification (NER)
ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")

def extract_transformer_entities(text):
    entities = ner_pipeline(text)
    return [(entity['word'], entity['entity']) for entity in entities]

df['transformer_entities'] = df['job_description'].apply(extract_transformer_entities)


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


RuntimeError: At least one of TensorFlow 2.0 or PyTorch should be installed. To install TensorFlow 2.0, read the instructions at https://www.tensorflow.org/install/ To install PyTorch, read the instructions at https://pytorch.org/.